In [11]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
plt.style.use('seaborn')

import mtt
from os.path import join, realpath
from os import getcwd
sbfile = join(realpath(getcwd()),'three_steps.sb')
wiring = mtt.Wiring.fromFile2(sbfile, default_margin=1000., input_val=10., current_assignment='maxKD')
wiring

In [12]:
model = mtt.MTT(wiring)
model.draw(3)

# Digitized current config

In [13]:
# for block in wiring.blocks:
#     print(block.getDigitizedParameterString(model))

# Simulation

In [14]:
sbml = model.toSBML()
# print(sbml)
from roadrunner import RoadRunner
with open('out-sbml.xml', 'w') as f:
    f.write(sbml)
xmod = RoadRunner(sbml)
xmod.reset()
# simulate over transient
xmod.simulate(0,5,1000,selections=['time']+['X','Y','Z','PX','PY','PZ'])
xmod.simulate(5,10,1000,selections=['time']+['X','Y','Z','PX','PY','PZ'])
xmod.plot()

In [15]:
from scipy.io import loadmat
from tellurium import plot, show
from numpy import vstack, correlate, amax, argmax, zeros, linspace, sum
from math import sqrt

In [16]:
rna_data = loadmat('data/rna.mat')
rna1X = rna_data['rna1X'].flatten()
rna1Y = rna_data['rna1Y'].flatten()
rna2X = rna_data['rna2X'].flatten()
rna2Y = rna_data['rna2Y'].flatten()
rna3X = rna_data['rna3X'].flatten()
rna3Y = rna_data['rna3Y'].flatten()
rna_x = vstack((rna1X,rna2X,rna3X)).T
chip_duration = float(rna_x[-1,0] - rna_x[0,0])
rna_y = vstack((rna1Y,rna2Y,rna3Y)).T

In [17]:
protein_data = loadmat('data/protein.mat')
Protein1X = protein_data['Protein1X'].flatten()
Protein1Y = protein_data['Protein1Y'].flatten()
Protein2X = protein_data['Protein2X'].flatten()
Protein2Y = protein_data['Protein2Y'].flatten()
Protein3X = protein_data['Protein3X'].flatten()
Protein3Y = protein_data['Protein3Y'].flatten()
protein_x = vstack((Protein1X,Protein2X,Protein3X)).T
protein_y = vstack((Protein1Y,Protein2Y,Protein3Y)).T

In [18]:
plot(rna1X, rna1Y, name='X', show=False)
plot(rna2X, rna2Y, name='Y', show=False)
plot(rna3X, rna3Y, name='Z', show=False)
show()

In [19]:
plot(Protein1X, Protein1Y, name='PX', show=False)
plot(Protein2X, Protein2Y, name='PY', show=False)
plot(Protein3X, Protein3Y, name='PZ', show=False)
show()

In [20]:
N = 100

max_rna_cor = zeros((3,N))
max_rna_index = zeros((3,N))
max_rna_time = zeros((3,N))

max_protein_cor = zeros((3,N))
max_protein_index = zeros((3,N))
max_protein_time = zeros((3,N))

ratio_range = linspace(2., 10., N)
for n,t_ratio in enumerate(ratio_range):
    sim_duration = t_ratio*chip_duration
    xmod.reset()
    # simulate over transient
    xmod.simulate(0,5,1000,selections=['time']+['X','Y','Z','PX','PY','PZ'])
    r = xmod.simulate(5.,5.+sim_duration,480,selections=['time']+['X','Y','Z','PX','PY','PZ'])
    
    sim_rna = r[:,1:4]
    sim_protein = r[:,4:7]
    
    for k,name in enumerate(('X corr.','Y corr.','Z corr.')):
        rna_corr = correlate(sim_rna[:,k],rna_y[:,k],mode='same') / \
            sqrt(sum(sim_rna[:,k]**2)*sum(rna_y[:,k]**2))
        max_rna_cor[k,n] = amax(rna_corr)
        max_rna_index[k,n] = argmax(rna_corr)
        max_rna_time[k,n] = r[int(max_rna_index[k,n]),0]
#         plot(r[:,0], rna_corr, name=name, xtitle = 'time (s)', ytitle='RNA corr. max = '+str(max_rna_time[0]), show=False)
#     show()
    for k,name in enumerate(('PX corr.','PY corr.','PZ corr.')):
        protein_corr = correlate(sim_protein[:,k],protein_y[:,k],mode='same') / \
            sqrt(sum(sim_protein[:,k]**2)*sum(protein_y[:,k]**2))
        max_protein_cor[k,n] = amax(protein_corr)
        max_protein_index[k,n] = argmax(protein_corr)
        max_protein_time[k,n] = r[int(max_protein_index[k,n]),0]
#         plot(r[:,0], protein_corr, name=name, xtitle = 'time (s)', ytitle='Protein corr. max t = '+str(max_protein_time[0]), show=False)
#     show()


nrows = 1
ncols = 1
fig,axes = plt.subplots(nrows, ncols, sharex='col', sharey='row', figsize=(8,6))

row = col = 0
for k,name in enumerate(('X','Y','Z')):
#     plot(ratio_range, max_rna_cor[k,:], name=name, xtitle = 'time ratio (digital/analog)', ytitle='Max RNA rel. corr.', show=False)
    a = axes
    a.set_title('Digital/Analog Correlation', fontsize=14)
    a.set_xlabel('time ratio (digital/analog)', fontsize=14)
    a.set_ylabel('Max rel. corr.', fontsize=14)
    a.plot(ratio_range, max_rna_cor[k,:], label=name, color='C'+str(1+k), linewidth=2)
# show()

col = 0
for k,name in enumerate(('PX','PY','PZ')):
#     plot(ratio_range, max_protein_cor[k,:], name=name, xtitle = 'time ratio (digital/analog)', ytitle='Max Protein rel. corr.', show=False)
    a = axes
    a.set_title('Digital/Analog Correlation', fontsize=14)
    a.set_xlabel('time ratio (digital/analog)', fontsize=14)
    a.set_ylabel('Max rel. corr.', fontsize=14)
    a.plot(ratio_range, max_protein_cor[k,:], label=name, color='C'+str(4+k), linewidth=2)
# show()
    handles, labels = a.get_legend_handles_labels()

fig.legend(handles, labels, loc='lower right')

plt.savefig('digital-analog-corr-plot.pdf', format='pdf')